In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
def load_and_resize_images_from_folder(folder, target_size):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, target_size)
            images.append(img)
    return np.array(images)

# Specify your image folder, target size, and batch size
image_folder = '/content/drive/MyDrive/REX Technologies/img_align_celeba'
target_size = (64, 64)

# Load and resize all images
X = load_and_resize_images_from_folder(image_folder, target_size)

# Normalize pixel values to the range [0, 1]
X = X / 255.0

label = pd.read_csv("/content/drive/MyDrive/REX Technologies/list_attr_celeba.csv")
y = np.array(label['Male'])
from sklearn.preprocessing import OneHotEncoder
encoder_y = OneHotEncoder()
y = encoder_y.fit_transform(y.reshape(-1,1)).toarray()
y = y[0:5000,]

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
# Create a neural network model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(64, 64, 3)),  # Input layer for resized images
    keras.layers.Dense(256, activation='relu'),           # Hidden layer with 256 units and ReLU activation
    keras.layers.Dense(128, activation='relu'),           # Hidden layer with 128 units and ReLU activation
    keras.layers.Dense(64, activation='relu'),            # Hidden layer with 64 units and ReLU activation
    keras.layers.Dense(2, activation='softmax')           # Output layer with 2 unit and sigmoid activation
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with batch learning
batch_size = 32
epochs = 50  # Adjust the number of epochs as needed

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print(f"Test Accuracy: {test_accuracy}")

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate additional evaluation metrics
print("Classification Report:")
print(classification_report(y_test.argmax(axis=1), y_pred_classes))

print("Confusion Matrix:")
print(confusion_matrix(y_test.argmax(axis=1), y_pred_classes))


Epoch 1/50
100/100 [==============================] - 6s 48ms/step - loss: 0.7047 - accuracy: 0.5472 - val_loss: 0.6765 - val_accuracy: 0.5913
Epoch 2/50
100/100 [==============================] - 7s 68ms/step - loss: 0.6853 - accuracy: 0.5678 - val_loss: 0.6778 - val_accuracy: 0.5975
Epoch 3/50
100/100 [==============================] - 6s 60ms/step - loss: 0.6896 - accuracy: 0.5578 - val_loss: 0.6813 - val_accuracy: 0.5950
Epoch 4/50
100/100 [==============================] - 4s 41ms/step - loss: 0.6858 - accuracy: 0.5669 - val_loss: 0.6849 - val_accuracy: 0.5675
Epoch 5/50
100/100 [==============================] - 5s 53ms/step - loss: 0.6853 - accuracy: 0.5644 - val_loss: 0.7034 - val_accuracy: 0.5962
Epoch 6/50
100/100 [==============================] - 7s 68ms/step - loss: 0.6833 - accuracy: 0.5731 - val_loss: 0.7283 - val_accuracy: 0.4038
Epoch 7/50
100/100 [==============================] - 5s 50ms/step - loss: 0.6809 - accuracy: 0.5741 - val_loss: 0.6844 - val_accuracy: 0.5638